In [ ]:
import whisper
from datasets import load_dataset
from torch import nn
import collections
import torch, gc

In [ ]:
# 학습된 모델 불러오기
whisper_model = whisper.load_model('large')  # model_name : large(default)

In [ ]:
# 데이터 불러오기
dataset_1 = load_dataset("anton-l/superb_demo", "er", split="session1")       # session1~5

In [ ]:
# test = []
# for data in dataset_1 :
#     gc.collect()
#     torch.cuda.empty_cache()    
#     print(data['file'])
#     audio = whisper.load_audio(data['file'])
#     edited_data = whisper.pad_or_trim(audio)
#     mel = whisper.log_mel_spectrogram(edited_data).to(whisper_model.device)
#     # dim 맞추기
#     mel = mel[None, :, :]       # torch.Size([1, 80, 3000])
#     print(mel)
#     output = whisper_model.encoder(mel)
    
#     test.append((output,data['label']))

---

In [ ]:
# encoder output에
# 평탄화(FC layer) -> 활성화함수 활성화 -> 분류기(softmax) 함수로 분류

In [2]:
import whisper
from datasets import load_dataset
from torch import nn
import collections
import torch, gc

/opt/conda/envs/lv3/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-01 05:55:29.982039: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-01 05:55:30.947563: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-01 05:55:30.947657: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dler

In [3]:
w_model = whisper.load_model('large')  # model_name : large(default)
encoder = w_model.encoder.to(torch.device("cuda"))

In [8]:
encoder

AudioEncoder(
  (conv1): Conv1d(80, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
  (blocks): ModuleList(
    (0): ResidualAttentionBlock(
      (attn): MultiHeadAttention(
        (query): Linear(in_features=1280, out_features=1280, bias=True)
        (key): Linear(in_features=1280, out_features=1280, bias=False)
        (value): Linear(in_features=1280, out_features=1280, bias=True)
        (out): Linear(in_features=1280, out_features=1280, bias=True)
      )
      (attn_ln): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (mlp): Sequential(
        (0): Linear(in_features=1280, out_features=5120, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=5120, out_features=1280, bias=True)
      )
      (mlp_ln): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
    (1): ResidualAttentionBlock(
      (attn): MultiHeadAttention(
        (query): Linear(in_feat

In [ ]:
# LayerNorm
# 각 인스턴스에 나온 feature을 모든 channel에 걸쳐 한번에 normalize하는 것
# CV에서는 Batch Norm, NLP에서는 LayerNorm을 표준으로 사용함
# >> NLP의 경우, 입력 길이의 가변성, 미니 배치 분포의 심한 변동성, RNN모델에 BN 개념의 모호성 때문에 LN

In [ ]:
my_classifier  = nn.Sequential(
    nn.Linear(1280, 512),   # FC 1
    nn.GELU(approximate='none'),
    nn.Linear(512, 7)        # FC 2
)

model = nn.Sequential(collections.OrderedDict([
    ('encoder', encoder),
    ('classifier', my_classifier)
]))

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
audio = whisper.load_audio(dataset_1['file'][0])
edited_data = whisper.pad_or_trim(audio)
mel = whisper.log_mel_spectrogram(edited_data).to(whisper_model.device)

In [ ]:
output = whisper_model.encoder(mel)

**test**

In [ ]:
test = []
for data in dataset_1 :
    gc.collect()
    torch.cuda.empty_cache()    
    print(data['file'])
    audio = whisper.load_audio(data['file'])
    edited_data = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(edited_data).to(whisper_model.device)
    # dim 맞추기
    mel = mel[None, :, :]       # torch.Size([1, 80, 3000])
    print(mel)
    output = whisper_model.encoder(mel)
    
    test.append((output,data['label']))
    
    

In [ ]:
test

---
test

In [ ]:
import whisper
import torch

input_file = '/opt/ml/final/honeybee.wav'
w_model = whisper.load_model('large')  # model_name : large(default)
w_model = model.to(torch.device("cuda"))
result = model.transcribe(input_file,fp16=False,language='English')

def split_wav(data, sample_rate, start, end):
    start *= sample_rate
    end *= sample_rate
    return data[start:end]

def split_run(input_file,time_list):
    audio = whisper.load_audio(input_file)  # 원본 wav 파일
    edited_audio = []
    
    for start,end,text in time_list :
        # start초 end초 까지의 데이터 추출  : array([0., 0., 0., ..., 0., 0., 0.], dtype=float32) 형태
        edited_audio.append(split_wav(audio, 16000, int(start), int(end)))
    return edited_audio

timeline = []
for line in result["segments"]:
    timeline.append([line['start'], line['end'],line['text']])          # [ start,end,text ] :str

audios = split_run(input_file, timeline)
audios
